In [1]:
import pandas as pd
from pathlib import Path
from os.path import normpath
import matplotlib.pyplot as plt
from ipywidgets import Dropdown, Output, Button, HBox

In [94]:
data_path = Path('data/cleaned')

In [95]:
sentiment_path = data_path / 'sentiment_all_cleaned.csv'

In [96]:
sentiment_path = sentiment_path.resolve(True)

In [97]:
top50_dim_path = data_path / 'top_50_dim.csv'

In [98]:
top50_dim_path = top50_dim_path.resolve(True)

In [7]:
sent_df = pd.read_csv(sentiment_path)

In [8]:
sent_df.head()

date                company  volume  positive  negative  sentiment  \
0  2010-01-01  cara_therapeutics_inc       0         0         0        0.5   
1  2010-01-02  cara_therapeutics_inc       0         0         0        0.5   
2  2010-01-03  cara_therapeutics_inc       0         0         0        0.5   
3  2010-01-04  cara_therapeutics_inc       0         0         0        0.5   
4  2010-01-05  cara_therapeutics_inc       0         0         0        0.5   

   esg_score  e_score  s_score  g_score  general_risk_score  esg_risk_score  \
0        5.0      5.0      5.0      5.0                 0.0             0.0   
1        5.0      5.0      5.0      5.0                 0.0             0.0   
2        5.0      5.0      5.0      5.0                 0.0             0.0   
3        5.0      5.0      5.0      5.0                 0.0             0.0   
4        5.0      5.0      5.0      5.0                 0.0             0.0   

   credit_risk_score  immediate_risk_score          ISIN  
0                0.0                   0.0  US1407551092  
1                0.0                   0.0  US1407551092  
2                0.0                   0.0  US1407551092  
3                0.0                   0.0  US1407551092  
4                0.0                   0.0  US1407551092

In [20]:
top50_dim_df = pd.read_csv(top50_dim_path)

In [21]:
top50_dim_df.head()

symbol                    shortName                     longName  \
0      MS               Morgan Stanley               Morgan Stanley   
1    TSLA                  Tesla, Inc.                  Tesla, Inc.   
2     BAC  Bank of America Corporation  Bank of America Corporation   
3  AIR.PA                    AIRBUS SE                    Airbus SE   
4     WFC        Wells Fargo & Company        Wells Fargo & Company   

         country             industry              sector currency  \
0  United States      Capital Markets  Financial Services      USD   
1  United States   Auto Manufacturers   Consumer Cyclical      USD   
2  United States  Banks - Diversified  Financial Services      USD   
3    Netherlands  Aerospace & Defense         Industrials      EUR   
4  United States  Banks - Diversified  Financial Services      USD   

  financialCurrency language region typeDisp     market fullExchangeName  \
0               USD    en-US     US   Equity  us_market             NYSE   
1               USD    en-US     US   Equity  us_market         NasdaqGS   
2               USD    en-US     US   Equity  us_market             NYSE   
3               EUR    en-US     US   Equity  fr_market            Paris   
4               USD    en-US     US   Equity  us_market             NYSE   

  exchange     company_sent  ticker  
0      NYQ   morgan_stanley      MS  
1      NMS     tesla_motors    TSLA  
2      NYQ  bank_of_america     BAC  
3      PAR           airbus  AIR.PA  
4      NYQ      wells_fargo     WFC

In [22]:
top50_sent_df = pd.merge(
    top50_dim_df[['company_sent', 'ticker', 'shortName']], 
    sent_df, how= 'left', 
    left_on= 'company_sent', 
    right_on= 'company',
    validate= 'one_to_many').drop('company_sent', axis= 1)

In [23]:
top50_sent_df.isna().sum(axis= 0)

ticker                    0
shortName                 0
date                      0
company                   0
volume                    0
positive                  0
negative                  0
sentiment                 0
esg_score               272
e_score                 272
s_score                 272
g_score                 272
general_risk_score      272
esg_risk_score          272
credit_risk_score       272
immediate_risk_score    272
ISIN                      0
dtype: int64

In [24]:
top50_sent_df.index = pd.to_datetime(top50_sent_df.date)

In [30]:
output_button = Output()
button = Button(description='Plot Volume')
drop = Dropdown(
    options= top50_dim_df.shortName,
    disabled=False,
)

def plot(button):
    with output_button:
        output_button.clear_output()
        top50_sent_df.query("shortName == @drop.value").volume.plot()
        plt.title(drop.value)
        plt.legend()
        plt.show()
        # output_button.append_display_data(plt.show())

button.on_click(plot)

display(HBox([button, drop]))
display(output_button)

Output()

In [60]:
top50_sent_df.groupby('company')[['volume', 'sentiment', 'esg_score']].agg(['mean', 'median', 'sum']).sort_values(('sentiment', 'mean'), ascending= False).head()

volume                sentiment            \
                              mean median     sum      mean    median   
company                                                                 
pepsico                  48.940949   34.0  253612  0.766802  0.800000   
nvidia                  109.029525   25.0  564991  0.761299  0.803309   
salesforce_com           69.479931   31.0  360045  0.759170  0.808669   
marriott_international   35.302200   27.0  182936  0.746948  0.800000   
comcast                  56.089927   41.0  290658  0.726565  0.769231   

                                    esg_score                          
                                sum      mean    median           sum  
company                                                                
pepsico                 3973.567695  6.411353  7.418525  33223.631743  
nvidia                  3945.052024  4.539410  4.799193  23523.224812  
salesforce_com          3934.018442  4.777107  4.894246  24754.966846  
marriott_international  3870.683604  4.921947  6.276529  25505.526845  
comcast                 3765.058802  4.824904  5.514663  25002.654377